In [79]:
import requests
import pandas as pd
import polars as pl
import zipfile
import io
import numpy as np
from datetime import date, datetime, timedelta

In [ ]:
#!pip install pandas sqlalchemy mysql-connector-python

In [82]:
url_zip = "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/CNES/cnes_estabelecimentos_csv.zip"
response = requests.get(url_zip)
response.raise_for_status()

zip_in_memory = io.BytesIO(response.content)

nome_csv = "cnes_estabelecimentos.csv"  

with zipfile.ZipFile(zip_in_memory) as zf:
        print(f"Extraindo '{nome_csv}' do ZIP...")
        
        df = pd.read_csv(nome_csv, sep=';', encoding="latin-1", dtype="str")


Extraindo 'cnes_estabelecimentos.csv' do ZIP...


In [86]:
df["CO_UNIDADE"] = df["CO_UNIDADE"].astype(str)
df.CO_CNES = df.CO_CNES.astype("Int64")
df['CO_UF'] = df['CO_UF'].astype("Int64")
df['CO_IBGE'] = df['CO_IBGE'].astype("Int64")
df['NU_CNPJ_MANTENEDORA'] = df['NU_CNPJ_MANTENEDORA'].astype("Int64")
df['CO_NATUREZA_ORGANIZACAO'] = df['CO_NATUREZA_ORGANIZACAO'].astype("Int64")
df['CO_NIVEL_HIERARQUIA'] = df['CO_NIVEL_HIERARQUIA'].astype("Int64")
df['CO_ATIVIDADE'] = df['CO_ATIVIDADE'].astype("Int64")
df['TP_UNIDADE'] = df['TP_UNIDADE'].astype("Int64")
df['CO_CEP'] = df['CO_CEP'].astype("Int64")
df['CO_TURNO_ATENDIMENTO'] = df['CO_TURNO_ATENDIMENTO'].astype("Int64")
df['NU_CNPJ'] = df['NU_CNPJ'].astype("Int64")
df['CO_NATUREZA_JUR'] = df['CO_NATUREZA_JUR'].astype("Int64")
df['CO_MOTIVO_DESAB'] = df['CO_MOTIVO_DESAB'].astype("Int64")
df["data_extracao"] = datetime.today()

In [88]:
lista_float = ['NU_LATITUDE', 'NU_LONGITUDE']

for i in lista_float:
    df[i] = df[i].astype(float)

condicoes_bool = [
    df['CO_AMBULATORIAL_SUS'] == "SIM",
    df['CO_AMBULATORIAL_SUS'] == "NAO"
]

valores_bool = [
    True,
    False
]

df['CO_AMBULATORIAL_SUS'] = np.select(condicoes_bool, valores_bool, default=np.nan)

lista_booleanos = ['ST_CENTRO_CIRURGICO', 'ST_CENTRO_OBSTETRICO', 'ST_CENTRO_NEONATAL',
       'ST_ATEND_HOSPITALAR', 'ST_SERVICO_APOIO', 'ST_ATEND_AMBULATORIAL', 'CO_AMBULATORIAL_SUS']

for i in lista_booleanos:
    df[i] = df[i].astype(bool)

In [89]:
display(df)

,CO_CNES,CO_UNIDADE,CO_UF,CO_IBGE,NU_CNPJ_MANTENEDORA,NO_RAZAO_SOCIAL,NO_FANTASIA,CO_NATUREZA_ORGANIZACAO,DS_NATUREZA_ORGANIZACAO,TP_GESTAO,...,CO_NATUREZA_JUR,ST_CENTRO_CIRURGICO,ST_CENTRO_OBSTETRICO,ST_CENTRO_NEONATAL,ST_ATEND_HOSPITALAR,ST_SERVICO_APOIO,ST_ATEND_AMBULATORIAL,CO_MOTIVO_DESAB,CO_AMBULATORIAL_SUS,data_extracao
0,19,2602900000019,26,260290,11294402000162,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,POLICLINICA DR JAMACI DE MEDEIROS,<NA>,NaN,M,...,1244,True,True,True,True,True,True,<NA>,True,2025-09-05 15:20:37.178066
1,27,2602900000027,26,260290,10930451000181,CASA DE SAUDE E MATERNIDADE SANTA HELENA LTDA,CASA DE SAUDE SANTA HELENA,<NA>,NaN,M,...,2062,True,True,True,True,True,True,<NA>,True,2025-09-05 15:20:37.178066
2,35,2602900000035,26,260290,11294402000162,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,HOSPITAL MENDO SAMPAIO,<NA>,NaN,M,...,1244,True,True,True,True,True,True,<NA>,True,2025-09-05 15:20:37.178066
3,43,2602900000043,26,260290,11294402000162,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,POLICLINICA DR MANUEL GOMES,<NA>,NaN,M,...,1244,True,True,True,True,True,True,<NA>,True,2025-09-05 15:20:37.178066
4,51,2602900000051,26,260290,11294402000162,PREFEITURA MUNICIPAL DO CABO DE SANTO AGOSTINHO,POLICLINICA VICENTE MENDES,<NA>,NaN,M,...,1244,True,True,True,True,True,True,<NA>,True,2025-09-05 15:20:37.178066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583893,9999930,4211909999930,42,421190,<NA>,POSTO COLETAS BIOLOGICAS PAGANI LTDA,SANTA LUZIA,<NA>,NaN,M,...,2062,True,True,True,True,True,True,<NA>,True,2025-09-05 15:20:37.178066
583894,9999949,3529009999949,35,352900,44477909000100,PREFEITURA MUNICIPAL DE MARILIA,USF JARDIM MARACA,<NA>,NaN,M,...,1244,True,True,True,True,True,True,<NA>,True,2025-09-05 15:20:37.178066
583895,9999957,5208609999957,52,520860,1065846000172,MUNICIPIO DE GOIANESIA,CENTRO DE ESPECIALIDADE ODONTOLOGICA CEO,<NA>,NaN,M,...,1244,True,True,True,True,True,True,<NA>,True,2025-09-05 15:20:37.178066
583896,9999965,3171309999965,31,317130,<NA>,RPS SERVICOS MEDICOS,RPS SERVICOS MEDICOS,<NA>,NaN,M,...,2232,True,True,True,True,True,True,<NA>,True,2025-09-05 15:20:37.178066


In [78]:
pagina = 0
df_agg = pd.DataFrame()
while True:
    url = f"https://apidadosabertos.saude.gov.br/cnes/estabelecimentos?limit=400&offset={pagina}"
    
    resp = requests.get(url)
    
    print(f"Página {pagina} - Status Code: {resp.status_code}")

    if resp.status_code == 502 or resp.status_code == 500:
        print("Erro na requisição")
        continue

    if resp.status_code != 200:
        print("Erro na requisição")
        break

    dados = resp.json()
    dados = dados.get("estabelecimentos", [])

    if not dados:
        print("Fim dos dados")
        break
    
    df = pd.DataFrame(dados)
    
    if not df.empty:
        df_agg = pd.concat([df_agg, df], ignore_index= True)
        print(f"Registros coletados: {len(df)}")

    if pagina == 1:
        break
    
    pagina += 1

Página 0 - Status Code: 200
Registros coletados: 20
Página 1 - Status Code: 200
Registros coletados: 20
